# DMML : Assignment 1
### Paarth Iyer - MCS202218

In [45]:
import numpy as np
import pandas as pd

np.random.seed(100)

Importing the packages used

In [46]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn import naive_bayes
from sklearn.preprocessing import LabelEncoder

# Question 1

bank_data loads the data from a ssv to a dataframe using pandas.

While making bank_X, bank_data is copied while dropping the columns 'y', 'duration' and 'day_of_week'

In [47]:
bank_data = pd.read_csv(r"./bank-data/bank-additional-full.csv",sep=';')
bank_X = bank_data.drop(columns = ['y', 'duration','day_of_week'])
bank_y = bank_data.y.values     # 'yes' or 'no' column, the expected output of model

THe columns 'housing' and 'loan' are label encoded such that 'yes' = 2, 'unknown' = 1 and 'no'= 0

In [48]:
# default has too much unknown
# hosuing and loan unknowns are 1to1
yn_rank = ['no','unknown','yes']
yn_encoder = LabelEncoder().fit(yn_rank)
for field in ['housing','loan'] : 
    bank_X[field] = yn_encoder.fit_transform(bank_X[field])

The 'education' and 'job' column are label encoded using the order below.

After that, rest of the categorical features are OneHot encoded

In [49]:
#Label encode education with accordance to their level
edu_rank = ["illiterate","basic.4y","basic.6y","basic.9y","unknown","high.school","university.degree","professional.course"]
bank_X["education"] = LabelEncoder().fit(edu_rank).fit_transform(bank_X.education)

#Label encode jobs with their distribution in their response
job_rank = ["admin.","blue-collar","technician","services","management","entrepreneur","self-employed","retired","housemaid","unemployed","student","unknown"]
bank_X["job"] = LabelEncoder().fit(edu_rank).fit_transform(bank_X.job)

bank_X = pd.get_dummies(bank_X) # onehot encoding rest of the categorical features

Seperating training and test data

In [50]:
bank_X_train, bank_X_test, bank_y_train, bank_y_test = sklearn.model_selection.train_test_split(bank_X, bank_y)

### Decision Tree
Decision Tree with a max depth of 17 and min sample split of 7

In [51]:
tree_clf = DecisionTreeClassifier(max_depth=15,min_samples_split = 7,criterion='gini')
tree_clf.fit(bank_X_train, bank_y_train)

DecisionTreeClassifier(max_depth=15, min_samples_split=7)

In [52]:
print(sklearn.metrics.classification_report(bank_y_test,tree_clf.predict(bank_X_test)))

              precision    recall  f1-score   support

          no       0.92      0.96      0.94      9149
         yes       0.50      0.31      0.38      1148

    accuracy                           0.89     10297
   macro avg       0.71      0.64      0.66     10297
weighted avg       0.87      0.89      0.88     10297



### Naive bayes
For Naive bayes, the GaussianNB is used

In [53]:
bayes = naive_bayes.GaussianNB().fit(bank_X_train, bank_y_train)

In [54]:
print(sklearn.metrics.classification_report(bank_y_test,bayes.predict(bank_X_test)))

              precision    recall  f1-score   support

          no       0.93      0.90      0.91      9149
         yes       0.35      0.45      0.39      1148

    accuracy                           0.85     10297
   macro avg       0.64      0.67      0.65     10297
weighted avg       0.86      0.85      0.85     10297



# Question 2


In [55]:
bolly_data = pd.read_csv(r"./bollywood-data/bollywood-movies.csv")

Creating bank_y by comparing the revenue and budget for each movie

In [56]:
bolly_y = ['hit' if i else 'flop' for i in bolly_data['Revenue(INR)'] > bolly_data['Budget(INR)']]

Creating bolly_X by dropping the Movie name as it will be unique for just about each movie.

As we want to predict budget or flop before the movie is released, Revenue is also dropped.

In [57]:
bolly_X = bolly_data.drop(columns = ['Movie Name', 'Revenue(INR)'])

#### Preprocessing X
All the 'yes'/'no' fields are label encoded with 'yes'=1 and 'no'=0. 

The 'Release Period' field is encoded with 'Normal' = 0 and 'Holiday' = 1. 

Rest of the columns are then OneHot Encoded

In [58]:

yn_rank = ['no','yes']
yn_encoder = LabelEncoder().fit(yn_rank)
for field in ['Whether Remake', 'Whether Franchise','New Actor', 'New Director', 'New Music Director'] : 
    bolly_X[field] = yn_encoder.fit_transform(bolly_X[field])

bolly_X['Release Period'] = LabelEncoder().fit(['Normal','Holiday']).fit_transform(bolly_X['Release Period'])

bolly_X = pd.get_dummies(bolly_X)

For the OneHot columns, we will drop the ones where the Lead star, Director or Music Director only have one or two movie under their name. 

(Counting the lead starts, Directors and Music Directors in the dataset using the Counter object.)

In [59]:
from collections import Counter
actors = Counter(bolly_data['Lead Star'])
direc = Counter(bolly_data['Director'])
mdirec = Counter(bolly_data['Music Director'])

In [60]:
print(actors.most_common(10))
print(direc.most_common(10))
print(mdirec.most_common(10))

[('Akshay Kumar', 48), ('Ajay Devgn', 35), ('Emraan Hashmi', 27), ('Salman Khan', 27), ('Amitabh Bachchan', 21), ('Saif Ali Khan', 18), ('Sanjay Dutt', 18), ('Shahid Kapoor', 18), ('John Abraham', 17), ('Ranbir Kapoor', 17)]
[('Ram Gopal Verma', 18), ('Vikram Bhatt', 16), ('Priyadarshan', 16), ('Suresh Jain', 15), ('Mohit Suri', 11), ('Rohit Shetty', 11), ('Anurag Kashyap', 10), ('Madhur Bhandarkar', 9), ('Anees Bazmee', 9), ('David Dhawan', 8)]
[('Pritam', 92), ('Himesh Reshammiya', 51), ('Sajid - Wajid', 48), ('Anu Malik', 42), ('Shankar - Ehsaan - Loy', 41), ('Vishal - Shekhar', 38), ('Salim - Sulaiman', 35), ('A.R. Rehman', 26), ('Amit Trivedi', 24), ('Sachin - Jigar', 24)]


In [61]:

for act in actors.keys():
    if actors[act] <3:
        bolly_X = bolly_X.drop('Lead Star_'+act, axis=1)

for act in direc.keys():
    if direc[act] <2:
        bolly_X = bolly_X.drop('Director_'+act, axis=1)

for act in mdirec.keys():
    if mdirec[act] <3:
        bolly_X = bolly_X.drop('Music Director_'+act, axis=1)

len(bolly_X.columns)

563

Creating training and test set

In [62]:
bolly_X_train, bolly_X_test, bolly_y_train, bolly_y_test = sklearn.model_selection.train_test_split(bolly_X, bolly_y, test_size=0.2)

### Decision Tree
Decision tree created with a max depth of 5, having atleast 3 samples at the leafs and using the gini criterion

In [63]:
tree_bolly = DecisionTreeClassifier(max_depth=5,min_samples_leaf=3,criterion='gini')
tree_bolly.fit(bolly_X_train, bolly_y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=3)

In [64]:
print(sklearn.metrics.classification_report(bolly_y_test,tree_bolly.predict(bolly_X_test)))

              precision    recall  f1-score   support

        flop       0.80      0.88      0.84        92
         hit       0.95      0.92      0.94       248

    accuracy                           0.91       340
   macro avg       0.88      0.90      0.89       340
weighted avg       0.91      0.91      0.91       340



### Naive Bayes
Creating a NB classifier using GaussionNB

In [65]:
bayes = naive_bayes.GaussianNB().fit(bolly_X_train, bolly_y_train)

In [66]:
print(sklearn.metrics.classification_report(bolly_y_test,bayes.predict(bolly_X_test)))

              precision    recall  f1-score   support

        flop       0.82      0.66      0.73        92
         hit       0.88      0.95      0.91       248

    accuracy                           0.87       340
   macro avg       0.85      0.81      0.82       340
weighted avg       0.87      0.87      0.87       340

